In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Split each image to four

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np

def split_image_matplotlib(image_path):
    img = mpimg.imread(image_path)
    basename = os.path.splitext(os.path.basename(image_path))[0]

    height, width = img.shape[0], img.shape[1]
    print(height, width)
    mid_y, mid_x = height // 2, width // 2
    print(mid_y, mid_x)
    # Define the four quadrants
    quadrants = [
        img[:mid_y, :mid_x],    # Top-left
        img[:mid_y, mid_x:],    # Top-right
        img[mid_y:, :mid_x],    # Bottom-left
        img[mid_y:, mid_x:],    # Bottom-right
    ]

    for i, quadrant in enumerate(quadrants, start=1):
        save_path = f"/content/drive/MyDrive/פרוייקט זיהוי נבגים/תמונות של 4 דגימות שונות/split_images/images/{basename}_{i}.png"
        plt.imsave(save_path, quadrant)
        print(f"Saved: {save_path}")

In [ ]:
import glob
images_to_split = glob.glob('/content/drive/MyDrive/פרוייקט זיהוי נבגים/תמונות של 4 דגימות שונות/new samples/*')
for image in images_to_split:
  split_image_matplotlib(image)

# Convert the initial pickle to json

In [ ]:
import pickle
import json
import numpy as np

def pickle_to_json(pickle_file, json_file):
    with open(pickle_file, "rb") as f:
        data = pickle.load(f)

    # If it's a numpy array → convert to list
    if isinstance(data, np.ndarray):
        data = data.tolist()

    # If it's something nested (dicts/lists with numpy inside)
    def convert(o):
        if isinstance(o, np.ndarray):
            return o.tolist()
        if isinstance(o, (np.int32, np.int64)):
            return int(o)
        if isinstance(o, (np.float32, np.float64)):
            return float(o)
        raise TypeError(f"Object of type {type(o)} is not JSON serializable")

    with open(json_file, "w", encoding="utf-8") as f:
        json.dump({"points": data}, f, ensure_ascii=False, indent=4, default=convert)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import json

def plot_image_and_polygon(image_path, json_file, title):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    polygons = data["points"]   # list of polygons

    img = Image.open(image_path).convert("RGB")

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img)

    for poly in polygons:
        points = np.array(poly, dtype=float)
        if points.ndim == 2 and points.shape[1] == 2:  # valid [N,2]
            ax.plot(points[:, 0], points[:, 1], "-r", linewidth=2)
            ax.fill(points[:, 0], points[:, 1], alpha=0.3)

    ax.set_title(title)
    plt.show()


# Remove Marks

In [ ]:
import json
import numpy as np
from shapely.geometry import Point, Polygon

def filter_polygons(polygons_file, remove_file, output_file_filtered, output_file_dropped):
    # Load polygons
    with open(polygons_file, "r") as f:
        poly_data = json.load(f)

    # Polygons are stored under key "points"
    polygons = poly_data["points"]

    # Load circle centers
    with open(remove_file, "r") as f:
        circles = json.load(f)
    print(f"There are {len(circles)} marks to remove")

    filtered_polygons = []
    dropped_polygons = []
    for poly in polygons:
        polygon = Polygon(poly)
        keep = True
        for c in circles:
            point = Point(c["x"], c["y"])
            if polygon.contains(point):
                keep = False
                dropped_polygons.append(poly)
                break
        if keep:
            filtered_polygons.append(poly)

    # Save filtered polygons
    with open(output_file_filtered, "w", encoding="utf-8") as f:
        json.dump({"points": filtered_polygons}, f, indent=4, ensure_ascii=False)

    with open(output_file_dropped, "w", encoding="utf-8") as f:
        json.dump({"points": dropped_polygons}, f, indent=4, ensure_ascii=False)

    print(f"{len(dropped_polygons)} polygons are dropped.")
    print(f"Saved files to {output_file_filtered}, {output_file_dropped}")




# Add Marks

In [ ]:
import json
import math

def create_circle_polygon(x, y, radius=5, num_points=20):
    """Generate a list of [x, y] points approximating a circle."""
    return [[x + radius * math.cos(2 * math.pi * i / num_points),
             y + radius * math.sin(2 * math.pi * i / num_points)]
            for i in range(num_points)]

def add_circles_to_polygons(polygons_file, add_file, output_file, radius=5, num_points=20):
    # Load polygons
    with open(polygons_file, "r") as f:
        poly_data = json.load(f)

    polygons = poly_data.get("points", [])

    # Load circles to add
    with open(add_file, "r") as f:
        circles = json.load(f)
    print(f"There are {len(circles)} marks to add")
    # Convert each circle into a polygon and append
    for c in circles:
        circle_poly = create_circle_polygon(c["x"], c["y"], radius=radius, num_points=num_points)
        polygons.append(circle_poly)

    # Save updated polygons
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump({"points": polygons}, f, indent=4, ensure_ascii=False)

    print(f"Saved updated polygons to {output_file}")



# Full pipeline

In [ ]:
import pickle
import json
import glob
pickle_files = glob.glob("/content/drive/MyDrive/פרוייקט זיהוי נבגים/תמונות של 4 דגימות שונות/split_images/pickles_new/*.pkl")
for pickle_file in pickle_files:

  json_file = pickle_file.replace("pickles_new","images").replace(".pkl",".json")
  image_file = pickle_file.replace("pickles_new","images").replace(".pkl",".png")
  #Convert the polygon to json
  pickle_to_json(pickle_file, json_file)
  plot_image_and_polygon(image_file, json_file, 'SAM tags')
  #Remove marks
  marks_to_remove = json_file.replace('/images/','/contour_mask_jsons/').replace('.json', '_contour_mask.json')
  filter_polygons(json_file, marks_to_remove, "/content/filtered_points.json", "/content/dropped_points.json")
  plot_image_and_polygon(image_file, "/content/filtered_points.json", 'After remove marks')
  ###plot_image_and_polygon(image_file,  "/content/dropped_points.json", 'Marks were removed')
  #Add marks
  marks_to_add = json_file.replace('/images/','/background_mask_jsons/').replace('.json', '_background_mask.json')
  final_tag = json_file.replace('/images/','/final_tags/')
  add_circles_to_polygons("/content/filtered_points.json", marks_to_add, final_tag, radius=20, num_points=40)
  plot_image_and_polygon(image_file, final_tag, 'Final tag')

  #break

# Full pipeline after thier tag

In [ ]:
import pickle
import json
import glob
json_files = glob.glob("/content/drive/MyDrive/פרוייקט זיהוי נבגים/תמונות של 4 דגימות שונות/split_images/first_manual_tagging/final_tags/*.json")
for json_file in json_files:
  file_name = json_file.split('/')[-1].split('.')[0]
  image_file = json_file.replace("first_manual_tagging/final_tags","images").replace(".json",".png")
  #Convert the polygon to json
  plot_image_and_polygon(image_file, json_file, 'SAM tags')
  #Remove marks
  marks_to_remove = f"/content/drive/MyDrive/פרוייקט זיהוי נבגים/JSONs/mark to remove/{file_name}_contour_masked.json"
  filter_polygons(json_file, marks_to_remove, "/content/filtered_points.json", "/content/dropped_points.json")
  plot_image_and_polygon(image_file, "/content/filtered_points.json", 'After remove marks')
  ###plot_image_and_polygon(image_file,  "/content/dropped_points.json", 'Marks were removed')
  #Add marks
  marks_to_add = f"/content/drive/MyDrive/פרוייקט זיהוי נבגים/JSONs/mark to add/{file_name}_background_masked.json"
  final_tag = f"/content/drive/MyDrive/פרוייקט זיהוי נבגים/תמונות של 4 דגימות שונות/split_images/After_Einav_tagging/{file_name}.json"
  add_circles_to_polygons("/content/filtered_points.json", marks_to_add, final_tag, radius=20, num_points=40)
  plot_image_and_polygon(image_file, final_tag, 'Final tag')

  #break